In [29]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf

In [30]:
path = 'lung_colon_image_set/lung_image_sets'

IMG_SIZE = 256
EPOCHS = 10
BATCH_SIZE = 64
SEED = 42

train_ds = tf.keras.utils.image_dataset_from_directory(
    path,
    labels='inferred',
    label_mode='int',
    batch_size = BATCH_SIZE,
    image_size = (IMG_SIZE, IMG_SIZE),
    validation_split = 0.2, 
    subset = 'training',
    seed = SEED,
    shuffle = True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    path,
    labels='inferred',
    label_mode= 'int',
    batch_size = BATCH_SIZE,
    image_size = (256, 256),
    validation_split=0.2,
    subset='validation',
    seed = SEED,
    shuffle = True
)

class_names = train_ds.class_names
print(class_names)

Found 15000 files belonging to 3 classes.
Using 12000 files for training.
Found 15000 files belonging to 3 classes.
Using 3000 files for validation.
['lung_aca', 'lung_n', 'lung_scc']


In [31]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
print(len(base_model.layers))

175


In [32]:
base_model.trainable = False

In [33]:
from tensorflow.keras import layers, models

In [34]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.BatchNormalization(),
        layers.Dense(3, activation='softmax')
        ])

In [35]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [36]:
def add_preprocess(x, y):
    x = tf.cast(x, tf.float32)
    return preprocess_input(x), y


train_ds = (train_ds.map(add_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
            .prefetch(tf.data.AUTOTUNE))
val_ds = (val_ds.map(add_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
          .prefetch(tf.data.AUTOTUNE))

In [37]:
model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [38]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint

tensorboard_callback = TensorBoard(log_dir='logs', histogram_freq=1)

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_accuracy') > 0.90:
            print('\n Validation accuracy has reached upto 90% so, stopping further training')
            self.model.stop_training = True

es = EarlyStopping(
    monitor = 'val_loss', 
    restore_best_weights=True
)

ler = ReduceLROnPlateau(
    monitor = 'val_loss',
    patience=2, 
    factor=0.5, 
    verbose=1
)

mc = ModelCheckpoint(
    "best.keras",
    save_best_only=True,
    monitor='val_loss'
)

callbacks = [
    es,            
    mc,            
    myCallback(),  
    ler 
 ]

In [39]:
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks)

Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9168 - loss: 0.1986
 Validation accuracy has reached upto 90% so, stopping further training
188/188 ━━━━━━━━━━━━━━━━━━━━ 810s 4s/step - accuracy: 0.9550 - loss: 0.1110 - val_accuracy: 0.9720 - val_loss: 0.0657 - learning_rate: 0.0010


In [ ]:
epochs = range(1, len(history.history['loss']) + 1)

plt.figure(figsize=(8,5))
plt.plot(epochs, history.history['loss'], marker='o', label='Train loss')
plt.plot(epochs, history.history['val_loss'], marker='o', label='Val loss')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.show()

plt.figure(figsize=(8,5))
plt.plot(epochs, history.history['accuracy'], marker='o', label='Train acc')
plt.plot(epochs, history.history['val_accuracy'], marker='o', label='Val acc')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.show()

Fine Tuning

In [41]:
base_model.trainable = True

for layer in base_model.layers[:100]:
    layer.trainable = False

In [43]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # smaller LR
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks
)


Epoch 1/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9417 - loss: 0.1567
 Validation accuracy has reached upto 90% so, stopping further training
188/188 ━━━━━━━━━━━━━━━━━━━━ 1420s 7s/step - accuracy: 0.9617 - loss: 0.1025 - val_accuracy: 0.9917 - val_loss: 0.0290 - learning_rate: 1.0000e-05


In [ ]:
plt.figure(figsize=(8,5))
plt.plot(history_finetune.history["val_loss"], label='Validation loss')
plt.plot(history_finetune.history['loss'], label='Train loss')
plt.xlabel("Epoch")
plt.ylabel("Validation Loss")
plt.legend()
plt.show()